<a href="https://colab.research.google.com/github/Ankita-Rane/Apache-Beam-Pipeline/blob/main/Introductory/Branched_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!{'pip install apache-beam'}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 KB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.8/278.8 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.2/526.2 KB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=a2d475d2b53577939e568cf5debf0d6a36bb114532242f23564682e5ca821c45
  Stored in directory: /root/.cache/pip/wheels/07/35/78/e9004fa30578734db7f10e7a211605f3f0778d2bdde38a239d
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 

In [2]:
from google.colab import files
uploadd=files.upload()

Saving dept_data.txt to dept_data.txt


In [12]:
!{'mkdir -p data'}
!ls

data  dept_data.txt  sample_data


In [20]:
!cd data

In [45]:
!ls /data/*

/data/Acc-00000-of-00001  /data/merge_op-00000-of-00001
/data/HR-00000-of-00001


In [43]:
!rm -rf /data/*

#Branched Pipeline
labelling must in branched pipeline or it starts giving error for similar transforms used multiple times. Unique labeling makes it different and allows parallel runs

Error throws - transform already exists in code

In [48]:
#creating steps by applying transforms to PCollection
import apache_beam as beam

p1=beam.Pipeline()

def SplitRow(element):
  return element.split(',')

input_collection=(
    p1 
    |beam.io.ReadFromText('./dept_data.txt')
    #used Map Transform to get output as list as it is 1-1 mapping
    |beam.Map(SplitRow)
)

HR_attendance_count=(
    input_collection
    |"filter HR" >> beam.Filter(lambda record:record[3]=='HR')
    |"Map HR" >> beam.Map(lambda record: ('HR_'+record[1],1))
    |"Combine_HR" >> beam.CombinePerKey(sum)
    #|"Write to HR" >> beam.io.WriteToText('/data/HR')
)

Account_attendance_count=(
    input_collection
    |"filter Accounts" >>beam.Filter(lambda record:record[3]=='Accounts')
    |"Map Accounts" >> beam.Map(lambda record: ('Accounts_'+record[1],1))
    |"Combine_Accounts" >> beam.CombinePerKey(sum)
    #|"Write to Accounts" >>beam.io.WriteToText('/data/Acc')
)

merge_output=(
    #tuple of PCollection
    (HR_attendance_count,Account_attendance_count)
    |"Merger of outputs">>beam.Flatten() #flatten works like union in SQL so both PCollection in tuple must have same number of columns and data types
    #Flatten transforms gets applied on tuple of Pcollections
    |"Write to merge">>beam.io.WriteToText('/data/merge_op')
)

p1.run()

#reading file using linux command

# !{(head -n 5 /data/HR*)}
# !{(head -n 5 /data/Acc*)}

!{(head -n 10 /data/merge_op-*)}

('HR_Beryl', 62)
('HR_Olga', 31)
('HR_Leslie', 31)
('HR_Mindy', 31)
('HR_Vicky', 31)
('HR_Richard', 31)
('HR_Kirk', 31)
('HR_Kaori', 31)
('HR_Oscar', 31)
('Accounts_Marco', 31)
